In [1]:
import urllib
from IPython.display import Markdown as md

### change to reflect your notebook
_nb_loc = "05_create_dataset/05_split_tfrecord.ipynb"
_nb_title = "Splitting dataset and writing TF Records"

_icons=["https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png", "https://www.tensorflow.org/images/colab_logo_32px.png", "https://www.tensorflow.org/images/GitHub-Mark-32px.png", "https://www.tensorflow.org/images/download_logo_32px.png"]
_links=["https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?" + urllib.parse.urlencode({"name": _nb_title, "download_url": "https://github.com/takumiohym/practical-ml-vision-book-ja/raw/master/"+_nb_loc}), "https://colab.research.google.com/github/takumiohym/practical-ml-vision-book-ja/blob/master/{0}".format(_nb_loc), "https://github.com/takumiohym/practical-ml-vision-book-ja/blob/master/{0}".format(_nb_loc), "https://raw.githubusercontent.com/takumiohym/practical-ml-vision-book-ja/master/{0}".format(_nb_loc)]
md("""<table class="tfo-notebook-buttons" align="left"><td><a target="_blank" href="{0}"><img src="{4}"/>Run in Vertex AI Workbench</a></td><td><a target="_blank" href="{1}"><img src="{5}" />Run in Google Colab</a></td><td><a target="_blank" href="{2}"><img src="{6}" />View source on GitHub</a></td><td><a href="{3}"><img src="{7}" />Download notebook</a></td></table><br/><br/>""".format(_links[0], _links[1], _links[2], _links[3], _icons[0], _icons[1], _icons[2], _icons[3]))

<table class="tfo-notebook-buttons" align="left"><td><a target="_blank" href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?name=Splitting+dataset+and+writing+TF+Records&download_url=https%3A%2F%2Fgithub.com%2Ftakumiohym%2Fpractical-ml-vision-book-ja%2Fraw%2Fmaster%2F05_create_dataset%2F05_split_tfrecord.ipynb"><img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/>Run in Vertex AI Workbench</a></td><td><a target="_blank" href="https://colab.research.google.com/github/takumiohym/practical-ml-vision-book-ja/blob/master/05_create_dataset/05_split_tfrecord.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a></td><td><a target="_blank" href="https://github.com/takumiohym/practical-ml-vision-book-ja/blob/master/05_create_dataset/05_split_tfrecord.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td><td><a href="https://raw.githubusercontent.com/takumiohym/practical-ml-vision-book-ja/master/05_create_dataset/05_split_tfrecord.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a></td></table><br/><br/>

# データセットの分割とTFRecordsの書き込み  

このノートブックでは、データセットを学習用、検証用、テスト用に分割し、それらの画像をTensorFlow Recordsファイルに書き込む方法を示します。  


Colabを利用する場合、以下のコマンドでアカウントの認証、環境設定を行う必要があります。<br>
実行には時間がかかりますので、可能であればVertex AI Workbenchでの実行をおすすめします。

In [2]:
!pip install -q apache-beam==2.38.0

In [ ]:
import os

import numpy as np
import pandas as pd

import apache_beam as beam
import tensorflow as tf


Colab環境を利用する場合、以下を実行して必要なモジュールのインストール、ファイルのダウンロードを行ってください。

In [ ]:
IS_COLAB_BACKEND = 'COLAB_RELEASE_TAG' in os.environ  # this is always set on Colab
if IS_COLAB_BACKEND:
    from google.colab import auth
    auth.authenticate_user()

    # Install dependencies
    !pip install gcsfs
    !pip install apache-beam[gcp]

    # Get files
    BASE_PATH = "https://raw.githubusercontent.com/takumiohym/practical-ml-vision-book-ja/main/05_create_dataset"
    !wget {BASE_PATH}/jpeg_to_tfrecord.py
    !wget {BASE_PATH}/run_dataflow.sh
    !sudo chmod 755 jpeg_to_tfrecord.py
    !sudo chmod 755 run_dataflow.sh

以下のセルの`PROJECT`にGoogle CloudのプロジェクトID、`BUCKET`に利用可能なGoogle Cloud Storageバケット名を入力してください。

また、使用できるバケットがない場合は、以下の三行目のコマンドをコメントアウトし、gsutil mbコマンドを実行して作成してください。

In [ ]:
PROJECT = "PROJECT_NAME"
BUCKET = "BUCKET_NAME"
# !gsutil mb -l us-central1 -p {PROJECT} gs://{BUCKET} 

%env PROJECT=$PROJECT
%env BUCKET=$BUCKET

## データの確認

In [5]:
df = pd.read_csv('gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/all_data.csv', names=['image','label'])
df.head()

,image,label
0,gs://practical-ml-vision-book/flowers_5_jpeg/f...,daisy
1,gs://practical-ml-vision-book/flowers_5_jpeg/f...,daisy
2,gs://practical-ml-vision-book/flowers_5_jpeg/f...,daisy
3,gs://practical-ml-vision-book/flowers_5_jpeg/f...,daisy
4,gs://practical-ml-vision-book/flowers_5_jpeg/f...,daisy


## データのランダムな分割

In [6]:
np.random.seed(10)
rnd = np.random.rand(len(df))
train = df[ rnd < 0.8  ]
valid = df[ (rnd >= 0.8) & (rnd < 0.9) ]
test  = df[ rnd >= 0.9 ]
print(len(df), len(train), len(valid), len(test))

3670 2930 359 381


In [7]:
%%bash
rm -rf output
mkdir output

In [8]:
train.to_csv('output/train.csv', header=False, index=False)
valid.to_csv('output/valid.csv', header=False, index=False)
test.to_csv('output/test.csv', header=False, index=False)

In [9]:
!head output/test.csv

gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/10466290366_cc72e33532.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/10712722853_5632165b04.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/11642632_1e7627a2cc.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/13583238844_573df2de8e_m.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/1374193928_a52320eafa.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/13953307149_f8de6a768c_m.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/14471433500_cdaa22e3ea_m.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/14523675369_97c31d0b5b.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/163978992_8128b49d3e_n.jpg,daisy
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/16401288243_36112bd52f_m.jpg,daisy


## Apache Beamを使用したTFレコードの書き込み  
ここでは、Apache Beamを利用して、jpgの画像データとラベルを読み込み、TFRecords形式に書き込む方法を確認します。

まずはローカルで実行するため、速度を優先して、5つのレコードだけを書き込んでみましょう。

In [10]:
outdf = test.head()
len(outdf)

5

In [11]:
outdf.values

array([['gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/10466290366_cc72e33532.jpg',
        'daisy'],
       ['gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/10712722853_5632165b04.jpg',
        'daisy'],
       ['gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/11642632_1e7627a2cc.jpg',
        'daisy'],
       ['gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/13583238844_573df2de8e_m.jpg',
        'daisy'],
       ['gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/1374193928_a52320eafa.jpg',
        'daisy']], dtype=object)

In [12]:
!gsutil cat gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dict.txt

daisy
dandelion
roses
sunflowers
tulips


In [13]:
with tf.io.gfile.GFile('gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dict.txt', 'r') as f:
    LABELS = [line.rstrip() for line in f]
print('Read in {} labels, from {} to {}'.format(
    len(LABELS), LABELS[0], LABELS[-1]))

2022-12-10 22:42:35.221523: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-10 22:42:35.221593: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Read in 5 labels, from daisy to tulips


In [14]:
def _string_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode('utf-8')]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def read_and_decode(filename):
    IMG_CHANNELS = 3
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=IMG_CHANNELS)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img

def create_tfrecord(filename, label, label_int):
    print(filename)
    img = read_and_decode(filename)
    dims = img.shape
    img = tf.reshape(img, [-1]) # flatten to 1D array
    return tf.train.Example(features=tf.train.Features(feature={
        'image': _float_feature(img),
        'shape': _int64_feature([dims[0], dims[1], dims[2]]),
        'label': _string_feature(label),
        'label_int': _int64_feature([label_int])
    })).SerializeToString()

with beam.Pipeline() as p:
    (p 
     | 'input_df' >> beam.Create(outdf.values)
     | 'create_tfrecord' >> beam.Map(lambda x: create_tfrecord(x[0], x[1], LABELS.index(x[1])))
     | 'write' >> beam.io.tfrecordio.WriteToTFRecord('output/train')
    )

gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/10466290366_cc72e33532.jpg


2022-12-10 22:42:41.318424: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-10 22:42:41.318473: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-10 22:42:41.318498: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-direct): /proc/driver/nvidia/version does not exist
2022-12-10 22:42:41.318851: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/10712722853_5632165b04.jpg
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/11642632_1e7627a2cc.jpg
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/13583238844_573df2de8e_m.jpg
gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/daisy/1374193928_a52320eafa.jpg


In [15]:
!ls -l output/train*

-rw-r--r-- 1 jupyter jupyter 8777320 Dec 10 22:52 output/train-00000-of-00001
-rw-r--r-- 1 jupyter jupyter  300932 Dec 10 22:42 output/train.csv


## Apache Beamでのデータの分割
Apache Beamを利用したデータ分割のサンプルコードを示します。ここではダミーデータを利用しています。<br>
`splits`で定義されたデータを、`train`と`valid`に分割して処理していることに注目してください。

In [16]:
def hardcoded(x, desired_split):
    split, rec = x
    print('hardcoded: ', split, rec, desired_split, split == desired_split)
    if split == desired_split:
        yield rec

with beam.Pipeline() as p:
        splits = (p
                  | 'input_df' >> beam.Create([
                      ('train', 'a'),
                      ('train', 'b'),
                      ('valid', 'c'),
                      ('valid', 'd')
                  ]))
        
        split = 'train'
        _ = (splits
                 | 'h_only_{}'.format(split) >> beam.FlatMap(
                     lambda x: hardcoded(x, 'train'))
         )        
        split = 'valid'
        _ = (splits
                 | 'h_only_{}'.format(split) >> beam.FlatMap(
                     lambda x: hardcoded(x, 'valid'))
        )

hardcoded:  train a train True
hardcoded:  train a valid False
hardcoded:  train b train True
hardcoded:  train b valid False
hardcoded:  valid c train False
hardcoded:  valid c valid True
hardcoded:  valid d train False
hardcoded:  valid d valid True


## Dataflowで実行  

Apache Beamコードは、Cloud Dataflowを使用してサーバーレスに実行できます。

[`./jpeg_to_tfrecord.py`](https://github.com/takumiohym/practical-ml-vision-book-ja/blob/main/05_create_dataset/jpeg_to_tfrecord.py) にCloud Dataflow用のコードが用意されています。<br>
beam.Pipeline() がクラウド上での実行用に以下のように書き換えられていることに注目してください。 

```
options = {
      'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
      'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
      'job_name': JOBNAME,
      'project': PROJECT,
      'teardown_policy': 'TEARDOWN_ALWAYS',
      'save_main_session': True
  }
opts = beam.pipeline.PipelineOptions(flags=[], **options)
with beam.Pipeline(RUNNER, options=opts) as p:
```

以下のコマンドで、Dataflowのジョブを実行します。

Google Cloudのコンソールから、Navigation Menu -> Dataflow -> preprocess-images-<timestamp> へ移動し、実行の様子を確認してください。

In [17]:
%%bash
python -m jpeg_to_tfrecord \
       --all_data gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/all_data.csv \
       --labels_file gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/dict.txt \
       --project_id $PROJECT \
       --output_dir gs://${BUCKET}/data/flower_tfrecords

2.8.2
Read in 5 labels, from daisy to tulips
Submitting DataflowRunner job: preprocess-images-221210-225839
Monitor at https://console.cloud.google.com/dataflow/jobs


2022-12-10 22:58:37.563592: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-10 22:58:37.563652: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CommandException: 1 files/objects could not be removed.


<img src="dataflow_pipeline.png" width="75%"/>

ジョブは30分ほどで完了します。

実行が完了したら、以下のセルで出力を確認してください。

In [ ]:
%%bash
gsutil ls -l gs://${BUCKET}/data/flower_tfrecords/*-00001-*

## License
Copyright 2022 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.